In [25]:
import os

from dotenv import load_dotenv
from etg_client import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

# Gemini API Key
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
GEMINI_REQUEST_TIMEOUT = 30.0

# Search Parameters
CITY = "Москва"
REGION_ID: int | None = None

CHECKIN_DATE = "2026-02-20"
CHECKOUT_DATE = "2026-02-23"

CURRENCY = "EUR"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": []}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Большая кровать, не было плесени. Не больше 10 мин пешком до метро. Отель находился рядом с центром"

# Reviews settings
REVIEWS_PER_SEGMENT = 30
REVIEWS_MAX_AGE_YEARS = 5
NEUTRAL_RATING_THRESHOLD = 7.0
NEGATIVE_RATING_THRESHOLD = 5.0

# Filters
MIN_PRICE: float | None = 0.0  # None = no minimum
MAX_PRICE: float | None = 150.0  # None = no maximum

In [26]:
from etg_client import (
    ETGClient,
    Hotel,
    HotelContent,
)

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [27]:
def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for r in regions[:5]:
        print(f"    - {r['name']} (type: {r['type']}, id: {r['id']})")

    return None

In [28]:
# Find region
region_id = REGION_ID or find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Москва'...
  [ETG] /api/b2b/v3/search/multicomplete/ - 200 in 0.23s
  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-02-20 to 2026-02-23
  Currency: EUR, Limit: 1000


In [29]:
import pandas as pd

# Search hotels
results = client.search_hotels_by_region(
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
)

hotels: list[Hotel] = results.get("hotels", [])
total_hotels = results.get("total_hotels", len(hotels))

if not hotels:
    print("No hotels found for the given criteria.")
    df_hotels = pd.DataFrame()
else:
    print(f"Found {len(hotels)} hotels (total available: {total_hotels})\n")

    # Create DataFrame
    hotels_data = []
    for hotel in hotels:
        rates = hotel.get("rates", [])
        if rates:
            first_rate = rates[0]
            payment_types = first_rate.get("payment_options", {}).get("payment_types", [])
            price = float(payment_types[0].get("show_amount", 0)) if payment_types else None
            room_name = first_rate.get("room_name", "")
            meal = first_rate.get("meal", "")
        else:
            price = None
            room_name = ""
            meal = ""
        
        hotels_data.append({
            "hotel_id": hotel["id"],
            "hid": hotel["hid"],
            "price": price,
            "currency": CURRENCY,
            "room": room_name[:40] if room_name else "",
            "meal": meal,
        })

    df_hotels = pd.DataFrame(hotels_data)
    df_hotels = df_hotels.sort_values("price", ascending=True).reset_index(drop=True)
    df_hotels.index += 1  # Start from 1

    # Display settings
    pd.set_option("display.max_colwidth", 50)
    pd.set_option("display.max_rows", 100)

df_hotels

  [ETG] /api/b2b/v3/search/serp/region/ - 200 in 4.38s
Found 857 hotels (total available: 857)



,hotel_id,hid,price,currency,room,meal
1,kapsulnyij_mini_hotelhostel_tri_vokzala,9741723,36.00,EUR,Двухместная Капсула без окна (общая ванн,nomeal
2,travel_inn_pervomajskaya_hostel,8877759,49.00,EUR,Двухместный номер Economy (общая ванная,nomeal
3,trevel_inn_alekseevskaya_hostel,8742666,49.00,EUR,Двухместный номер (без окна) Standard (о,nomeal
4,rus_aviator_hostel,8684414,51.00,EUR,Семейная капсула (общая ванная комната),nomeal
5,littlehotel_na_neopalimovskom_perulke_hostel,10151836,53.00,EUR,Двухместный номер Economy (общая ванная,nomeal
...,...,...,...,...,...,...
853,ararat_park_khaiatt,7599730,1295.34,EUR,Двухместный номер Парк (2 отдельные кров,nomeal
854,lotte_otel_moskva,7382812,1549.00,EUR,Двухместный номер Супериор (2 отдельные,nomeal
855,ritz_carlton,7597420,2098.58,EUR,Двухместный номер Супериор (двуспальная,nomeal
856,four_seasons_hotel_moscow,6374284,2103.00,EUR,Double номер Premier (двуспальная кроват,nomeal


In [30]:
from hotels import filter_hotels_by_price

# Apply price filter
hotels = filter_hotels_by_price(hotels, MIN_PRICE, MAX_PRICE)
print(f"After price filter: {len(hotels)} hotels")

After price filter: 298 hotels


In [31]:
from hotels import fetch_hotel_content

In [32]:
hotel_hids = [h["hid"] for h in hotels]

content_map = fetch_hotel_content(client, hotel_hids, LANGUAGE)

  [ETG] /api/content/v1/hotel_content_by_ids/ - 200 in 0.41s
  [ETG] /api/content/v1/hotel_content_by_ids/ - 200 in 0.50s
  [ETG] /api/content/v1/hotel_content_by_ids/ - 200 in 0.33s


In [33]:
content_data = []
for hid, content in content_map.items():
    content_data.append({
        "hid": hid,
        "name": content.get("name", "")[:40],
        "stars": content.get("star_rating", 0),
        "kind": content.get("kind", ""),
        "address": content.get("address", "")[:50],
        "latitude": content.get("latitude"),
        "longitude": content.get("longitude"),
        "check_in": content.get("check_in_time", ""),
        "check_out": content.get("check_out_time", ""),
    })

df_content = pd.DataFrame(content_data)

# Merge with prices from df_hotels
df_full = df_hotels.merge(df_content, on="hid", how="left")
df_full = df_full[["hotel_id", "name", "stars", "kind", "price", "meal", "address"]]
df_full = df_full.sort_values("price", ascending=True).reset_index(drop=True)
df_full.index += 1

df_full

,hotel_id,name,stars,kind,price,meal,address
1,kapsulnyij_mini_hotelhostel_tri_vokzala,Отель StayWay 3 Вокзала,0.0,Hotel,36.00,nomeal,"Новая Басманная улица, д. 10/1 помещ. 2/4, Москва"
2,travel_inn_pervomajskaya_hostel,Хостел Travel Inn Первомайская,0.0,Hostel,49.00,nomeal,"105077, г Москва, вн.тер.г. муниципальный окру..."
3,trevel_inn_alekseevskaya_hostel,Travel Inn Тимирязевская,0.0,Hotel,49.00,nomeal,"округ Тимирязевский, ул Ивановская, д. 23, пом..."
4,rus_aviator_hostel,Хостел Авиатор у Трех Вокзалов,0.0,Hostel,51.00,nomeal,"улица Ольховская, д.14, стр.5, Москва"
5,littlehotel_na_neopalimovskom_perulke_hostel,Меблированные комнаты Littlehotel,0.0,Hotel,53.00,nomeal,"Кутузовский проезд, 4 к 2, Москва"
...,...,...,...,...,...,...,...
853,ararat_park_khaiatt,NaN,NaN,NaN,1295.34,nomeal,NaN
854,lotte_otel_moskva,NaN,NaN,NaN,1549.00,nomeal,NaN
855,ritz_carlton,NaN,NaN,NaN,2098.58,nomeal,NaN
856,four_seasons_hotel_moscow,NaN,NaN,NaN,2103.00,nomeal,NaN


In [34]:
from reviews import fetch_reviews, filter_reviews, HotelReviewsFiltered

raw_reviews = fetch_reviews(client, hotel_hids, LANGUAGE)
reviews_map = filter_reviews(
    raw_reviews,
    max_age_years=REVIEWS_MAX_AGE_YEARS,
    reviews_per_segment=REVIEWS_PER_SEGMENT,
    neutral_threshold=NEUTRAL_RATING_THRESHOLD,
    negative_threshold=NEGATIVE_RATING_THRESHOLD,
)
print(f"Reviews: {sum(len(r['reviews']) for r in reviews_map.values())} filtered")

  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 2.40s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.91s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.66s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.31s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.18s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.13s
Reviews: 13041 filtered


In [35]:
# Create DataFrame with reviews summary
reviews_data = []
for hid, data in reviews_map.items():
    hotel_id = next((h["id"] for h in hotels if h["hid"] == hid), "")
    reviews_data.append({
        "hotel_id": hotel_id,
        "hid": hid,
        "total": data["total_reviews"],
        "positive": data["positive_count"],
        "neutral": data["neutral_count"],
        "negative": data["negative_count"],
    })

df_reviews = pd.DataFrame(reviews_data)
df_reviews = df_reviews.sort_values("total", ascending=False).reset_index(drop=True)
df_reviews.index += 1

# Merge with hotel info
df_reviews_full = df_reviews.merge(
    df_content[["hid", "name", "stars"]],
    on="hid",
    how="left"
)
df_reviews_full = df_reviews_full[["hotel_id", "name", "stars", "total", "positive", "neutral", "negative"]]


def show_reviews(hotel_id: str, segment: str = "all", limit: int = 5) -> None:
    """
    Show reviews for a hotel.
    
    Args:
        hotel_id: Hotel ID (e.g. 'rosewood_hong_kong')
        segment: 'positive', 'negative', 'neutral', or 'all'
        limit: Number of reviews to show per segment
    """
    hid = next((h["hid"] for h in hotels if h["id"] == hotel_id), None)
    if not hid:
        print(f"Hotel '{hotel_id}' not found")
        return
    
    data = reviews_map.get(hid)
    if not data:
        print(f"No reviews for hotel '{hotel_id}'")
        return
    
    hotel_name = content_map.get(hid, {}).get("name", hotel_id)
    print(f"{'='*60}")
    print(f"{hotel_name}")
    print(f"Total: {data['total_reviews']} | +{data['positive_count']} / ~{data['neutral_count']} / -{data['negative_count']}")
    print(f"{'='*60}\n")
    
    reviews = data["reviews"]
    
    def print_segment(name: str, filter_fn, limit: int):
        segment_reviews = [r for r in reviews if filter_fn(r)][:limit]
        if not segment_reviews:
            return
        print(f"--- {name} ({len(segment_reviews)}) ---")
        for r in segment_reviews:
            rating = r["rating"]
            date = r["created"][:10]
            lang = r.get("_lang", "?")
            plus = r.get("review_plus", "").strip()
            minus = r.get("review_minus", "").strip()
            print(f"\n[{rating}/10] {date} [{lang}]")
            if plus:
                print(f"  + {plus[:300]}")
            if minus:
                print(f"  - {minus[:300]}")
        print()
    
    if segment in ("all", "positive"):
        print_segment("POSITIVE", lambda r: r["rating"] >= NEUTRAL_RATING_THRESHOLD, limit)
    if segment in ("all", "neutral"):
        print_segment("NEUTRAL", lambda r: NEGATIVE_RATING_THRESHOLD <= r["rating"] < NEUTRAL_RATING_THRESHOLD, limit)
    if segment in ("all", "negative"):
        print_segment("NEGATIVE", lambda r: r["rating"] < NEGATIVE_RATING_THRESHOLD, limit)


df_reviews_full

,hotel_id,name,stars,total,positive,neutral,negative
0,otel_izmailovo_beta,Отель Измайлово Бета,3,3302,30,30,30
1,otel_izmailovo_gamma,Отель Измайлово Гамма,3,2563,30,30,30
2,katyusha_hotel_3,Отель Катюша,3,1175,30,30,23
3,sherston_okruzhnaya_apart_hotel,Апарт-отель Шерстон,3,991,30,30,30
4,gostinitsa_voskhod_,Меблированные комнаты Восход,0,821,30,19,30
...,...,...,...,...,...,...,...
291,studiya_na_prospekte_lenina_32v_326_flat,Квартира Студия на Проспекте Ленина 32А,0,1,1,0,0
292,s_vidom_na_odintsovo_apartments,Апартаменты C Видом на Одинцово,0,1,1,0,0
293,v_rasskazovke_v_stile_neoart_flat,Квартира в Рассказовке в стиле Нео-Арт,0,1,1,0,0
294,apartments_studio_apartonika,Апартаменты студия Апартоника,0,1,1,0,0


In [36]:
# Example: view reviews for a specific hotel
# show_reviews("four_seasons_st_petersburg")              # all segments, 5 per segment
# show_reviews("four_seasons_st_petersburg", "negative")  # only negative
# show_reviews("four_seasons_st_petersburg", "all", 10)   # all segments, 10 per segment

show_reviews("four_seasons_st_petersburg", limit=3)

Hotel 'four_seasons_st_petersburg' not found


In [37]:
class HotelCombined(Hotel, HotelContent):
    """Combined hotel data from search, content, and reviews."""
    reviews: HotelReviewsFiltered


def combine_hotel_data(
    hotels: list[Hotel],
    content_map: dict[int, HotelContent],
    reviews_map: dict[int, HotelReviewsFiltered],
) -> list[HotelCombined]:
    """Combine search results, content, and reviews."""
    print("Combining hotel data...")
    combined: list[HotelCombined] = []

    for hotel in hotels:
        hid = hotel["hid"]
        content = content_map.get(hid, {})
        reviews = reviews_map.get(hid, {
            "reviews": [],
            "total_reviews": 0,
            "positive_count": 0,
            "neutral_count": 0,
            "negative_count": 0,
        })

        combined.append({  # type: ignore[arg-type]
            **hotel,
            **content,
            "reviews": reviews,
        })

    print(f"  Combined {len(combined)} hotels")
    return combined

In [38]:
combined = combine_hotel_data(hotels, content_map, reviews_map)

Combining hotel data...
  Combined 298 hotels


In [39]:
import json

# Estimate token count for combined data
combined_json = json.dumps(combined, ensure_ascii=False)
char_count = len(combined_json)
# Rough estimate: ~4 chars per token for English, ~2-3 for Russian/mixed
estimated_tokens = char_count // 3

print(f"JSON size: {char_count:,} chars ({char_count / 1024 / 1024:.2f} MB)")
print(f"Estimated tokens: ~{estimated_tokens:,}")

JSON size: 20,273,615 chars (19.33 MB)
Estimated tokens: ~6,757,871


In [40]:
from hotels import presort_hotels
from scoring import score_hotels

In [43]:
# Pre-sort and limit to top 100 for LLM scoring
top_hotels = presort_hotels(combined, reviews_map, limit=100)
print(f"Pre-sorted: {len(top_hotels)} hotels for LLM scoring")

# Debug: show full prompts
DEBUG_SHOW_PROMPT = False

# Score hotels with progress
scoring_results = []
scoring_error = None

async for result in score_hotels(
    top_hotels,
    USER_PREFERENCES,
    currency=CURRENCY,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    batch_size=25,
):
    if result["type"] == "start":
        start = result["start"]
        print(f"\nScoring {start['total_hotels']} hotels in {start['total_batches']} batches (~{start['estimated_tokens']:,} tokens total)\n")
    elif result["type"] == "batch_start":
        bs = result["batch_start"]
        print(f"  → Batch {bs['batch']}/{bs['total_batches']}: {bs['hotels_in_batch']} hotels, ~{bs['estimated_tokens']:,} tokens")
        if DEBUG_SHOW_PROMPT:
            print(f"\n{'='*80}\nPROMPT (batch {bs['batch']}):\n{'='*80}\n{bs['prompt']}\n{'='*80}\n")
    elif result["type"] == "retry":
        retry = result["retry"]
        print(f"    ⚠ Retry {retry['attempt']}/{retry['max_attempts']}: {retry['error']}")
    elif result["type"] == "error":
        error = result["error"]
        scoring_error = error
        print(f"\n  ❌ ERROR [{error['error_type']}]: {error['message']}")
        break
    elif result["type"] == "progress":
        progress = result["progress"]
        print(f"  ✓ Batch {progress['batch']}/{progress['total_batches']} done ({progress['processed']}/{progress['total']} hotels)")
    elif result["type"] == "done":
        scoring_results = result["results"]

if scoring_error:
    print(f"\nScoring failed. Please check the error above.")
else:
    print(f"\nTotal scored: {len(scoring_results)} hotels")

Pre-sorted: 100 hotels for LLM scoring

Scoring 100 hotels in 4 batches (~127,365 tokens total)

  → Batch 1/4: 25 hotels, ~33,254 tokens
  ✓ Batch 1/4 done (25/100 hotels)
  → Batch 2/4: 25 hotels, ~32,096 tokens
  ✓ Batch 2/4 done (50/100 hotels)
  → Batch 3/4: 25 hotels, ~31,685 tokens
  ✓ Batch 3/4 done (75/100 hotels)
  → Batch 4/4: 25 hotels, ~30,329 tokens
  ✓ Batch 4/4 done (100/100 hotels)

Total scored: 99 hotels


In [44]:
from typing import Any
from hotels import get_ostrovok_url


def display_top_hotels(
    results: list[dict[str, Any]],
    hotels_data: list[dict[str, Any]],
    city: str,
    country_code: str = "DE",
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    # Build hotel_id -> hid mapping
    hid_map = {h.get("id", ""): h.get("hid", 0) for h in hotels_data}
    name_map = {h.get("id", ""): h.get("name", h.get("id", "")) for h in hotels_data}
    
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(results[:top_n], 1):
        hotel_id = hotel.get("hotel_id", "")
        hid = hid_map.get(hotel_id, 0)
        score = hotel.get("score", 0)
        name = name_map.get(hotel_id, hotel_id)
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        
        url = get_ostrovok_url(hotel_id, hid, city, country_code) if hid else ""

        # Print detailed info
        print(f"{i}. {name}")
        print(f"   Score: {score}/100")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   {url}")
        print()
        
        # Collect for DataFrame
        data.append({
            "name": name[:40],
            "score": score,
            "reasons": "; ".join(reasons[:2])[:80] if reasons else "",
            "penalties": "; ".join(penalties[:5])[:120] if penalties else "",
            "url": url,
        })
    
    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    total_found = len(hotels_data)
    selected = min(top_n, len(results))
    print(f"Всего найдено {total_found} отелей на эти даты. ")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


# Get country code from first hotel's region
first_hotel = combined[0] if combined else {}
region = first_hotel.get("region", {})
country_code = region.get("country_code", "DE")

pd.set_option("display.max_colwidth", 100)
display_top_hotels(scoring_results, combined, CITY, country_code, top_n=10)


TOP 10 HOTELS

1. Апарт-отель Norke Prime Сретенка
   Score: 95/100
   + В пешей доступности от центра и метро Сухаревская; Наличие больших двуспальных кроватей в студиях; Высокий рейтинг чистоты в отзывах
   - Отсутствие вентиляции и высокая влажность в некоторых номерах
   https://ostrovok.ru/hotel/russia/москва/mid10864367/apartotel_norke_prime_sretenskaya/

2. Мини-Отель Axel Hotel Moscow
   Score: 95/100
   + Находится в живописном центре города; 10 минут пешком до метро Чистые пруды; Есть номера с большой двуспальной кроватью
   https://ostrovok.ru/hotel/russia/москва/mid8880522/hotel_axel_moscow/

3. Хостел 5Небо
   Score: 95/100
   + Всего 1 км от центра города; 10 минут пешком до метро Китай-город; Есть номера с большой двуспальной кроватью
   https://ostrovok.ru/hotel/russia/москва/mid9990852/hostel_5nebo/

4. Отель Ист-Вест
   Score: 95/100
   + Находится в 1 км от центра города (Тверской бульвар); В наличии номера с большой двуспальной кроватью; Превосходное расположение д

,name,score,reasons,penalties,url
1,Апарт-отель Norke Prime Сретенка,95,В пешей доступности от центра и метро Сухаревская; Наличие больших двуспальных к,Отсутствие вентиляции и высокая влажность в некоторых номерах,https://ostrovok.ru/hotel/russia/москва/mid10864367/apartotel_norke_prime_sretenskaya/
2,Мини-Отель Axel Hotel Moscow,95,Находится в живописном центре города; 10 минут пешком до метро Чистые пруды,,https://ostrovok.ru/hotel/russia/москва/mid8880522/hotel_axel_moscow/
3,Хостел 5Небо,95,Всего 1 км от центра города; 10 минут пешком до метро Китай-город,,https://ostrovok.ru/hotel/russia/москва/mid9990852/hostel_5nebo/
4,Отель Ист-Вест,95,Находится в 1 км от центра города (Тверской бульвар); В наличии номера с большой,Нет явного подтверждения отсутствия плесени в описании,https://ostrovok.ru/hotel/russia/москва/mid7842381/east_west/
5,Отель Norke Prime Китай-город,92,Расположен в самом центре (Китай-город); 500 метров до метро (около 6 минут пешк,В отзывах упоминается отсутствие холодильника в номере,https://ostrovok.ru/hotel/russia/москва/mid9767561/norke_prime_kitajgorod_hotel/
6,Отель Винтерфелл Чистые Пруды,92,"Исторический центр, пешком до Кремля; Рядом три станции метро (Чистые пруды)",Очень маленькие номера (тесно),https://ostrovok.ru/hotel/russia/москва/mid9210214/vinterfell_na_chistyih_prudah_hotel/
7,Меблированные комнаты на Никитской (14),90,Идеальное расположение в самом центре Москвы; Большие кровати и чистые номера,Общий санузел для всех гостей; Высокая слышимость из-за тонких перегородок,https://ostrovok.ru/hotel/russia/москва/mid10163326/na_nikitskoy_11_living_quarters/
8,Апарт-Отель Norke Prime Зарядье,90,Центральное расположение рядом с парком Зарядье; 500 метров до метро Китай-город,В отзывах жалуются на сквозняки от окон зимой,https://ostrovok.ru/hotel/russia/москва/mid10163207/norke_prime_zaryadye_aparthotel/
9,Меблированные комнаты Nova на Белорусско,88,5 минут пешком до метро Белорусская; Большие удобные кровати и чистота в номерах,Общая ванная комната для бюджетных номеров,https://ostrovok.ru/hotel/russia/москва/mid13342151/nova_na_belorusskoy_mini_hotel/
10,Меблированные комнаты на Зубовском 3-2 Б,88,Всего 3 км от центра города; Очень удобные кровати по отзывам гостей,Общая ванная комната для стандартных номеров,https://ostrovok.ru/hotel/russia/москва/mid13371562/na_zubovskom_32_b_living_quarters/
